# Telecom X — Análisis de Evasión de Clientes (Churn)

**Fecha:** 2025-08-10

Este notebook está diseñado para ejecutarse en Google Colab. Contiene etapas de ETL, análisis exploratorio de datos (EDA), visualizaciones con `matplotlib` y un informe final con conclusiones y recomendaciones para reducir la evasión (churn).

Se cargará el dataset desde el siguiente enlace (raw):

- https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json

---


In [ ]:
# Código para ejecutar en Google Colab (o localmente)
# Si estás en Colab, descomenta la línea de instalación si necesitas paquetes adicionales.
# !pip install --upgrade pip

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['axes.grid'] = True

# Mostrar versiones
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', plt.__version__)


In [ ]:
# Cargar datos directamente desde el repositorio (raw URL)
import requests, io, json
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

resp = requests.get(url)
resp.raise_for_status()
data = resp.json()

# Convertir a DataFrame
df = pd.json_normalize(data)
print('Registros cargados:', len(df))
df.head()

In [ ]:
# Inspección inicial
df.info()
print('\nColumnas:\n', df.columns.tolist())
df.describe(include='all').T

In [ ]:
# --- Limpieza y Tratamiento de Datos (ETL) ---
df_clean = df.copy()

# 1) Normalizar nombres de columnas (sin espacios, todo lower)
df_clean.columns = [c.strip().lower().replace(' ', '_') for c in df_clean.columns]

# 2) Detectar y mostrar valores faltantes
missing = df_clean.isna().sum().sort_values(ascending=False)
missing[missing>0]

# 3) Intentar transformar columnas numéricas que estén como strings
for col in df_clean.columns:
    if df_clean[col].dtype == object:
        # intentar quitar símbolos y comas, convertir a num si tiene sentido
        try:
            df_clean[col] = df_clean[col].str.replace(',', '').str.replace('$','')
            df_clean[col] = pd.to_numeric(df_clean[col], errors='ignore')
        except Exception:
            pass

# 4) Detección de columnas tipo booleano (o con valores 'Yes'/'No', 'True'/'False')
for col in df_clean.select_dtypes(include='object').columns:
    uq = df_clean[col].dropna().unique()[:10]
    if set([str(x).lower() for x in uq]).issubset({'yes','no','true','false','y','n','0','1'}):
        df_clean[col] = df_clean[col].map(lambda x: str(x).strip().lower() if pd.notna(x) else x)
        df_clean[col] = df_clean[col].replace({'yes':1,'y':1,'true':1,'1':1,'no':0,'n':0,'false':0,'0':0})
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# 5) Mostrar resumen después de limpieza
print('Dimensiones antes:', df.shape, 'después:', df_clean.shape)
df_clean.head()

In [ ]:
# --- Feature engineering y visión general del Churn ---
df2 = df_clean.copy()

# Intentar identificar columna de churn (nombres comunes: churn, cancel, exited, is_churn)
churn_cols = [c for c in df2.columns if 'churn' in c or 'cancel' in c or 'exit' in c or 'churned' in c]
churn_cols

# Si no existe, buscar por valores binarios que representen churn
if not churn_cols:
    # heurística: columnas con 0/1 y nombre sospechoso
    possible = [c for c in df2.columns if df2[c].dropna().isin([0,1]).all() and ('status' in c or 'active' in c or 'estado' in c)]
    churn_cols = possible

churn_cols, (len(churn_cols)>0)

In [ ]:
# --- EDA: Tasa de Churn y distribuciones ---
# Definir columna churn si se detectó. Si no, pedir al usuario que la especifique.
if churn_cols:
    churn_col = churn_cols[0]
    print('Usando columna de churn detectada:', churn_col)
else:
    # si no se detecta, mostrar columnas para que el usuario elija manualmente
    print('No se detectó automáticamente una columna de churn. Revisa las columnas y asigna `churn_col` manualmente.')
    print('Columnas disponibles:', df2.columns.tolist())
    churn_col = None

if churn_col:
    # Asegurar tipo binario
    df2[churn_col] = pd.to_numeric(df2[churn_col], errors='coerce')
    churn_rate = df2[churn_col].mean()
    print(f'Tasa de churn (promedio de columna {churn_col}): {churn_rate:.4f} ({churn_rate*100:.2f}%)')
    
    # Tabla de frecuencia para churn por contrato (si existe contract)
    cat_cols = [c for c in df2.columns if df2[c].dtype=='object' or df2[c].nunique() < 30]
    print('\nColumnas categóricas principales a explorar:', cat_cols[:10])
    
    # Gráficos principales
    # 1) Pie o bar de churn vs no churn
    counts = df2[churn_col].value_counts(dropna=False)
    plt.figure(figsize=(6,4))
    counts.plot(kind='bar')
    plt.title('Distribución Churn (0 = no, 1 = yes)')
    plt.xlabel('Churn')
    plt.ylabel('Count')
    plt.show()
    
    # 2) Churn rate por columna categórica más relevante (ejemplo: contract_type, payment_method)
    for c in ['contract_type','contract','payment_method','gender','internet_service','senior_citizen']:
        if c in df2.columns:
            grp = df2.groupby(c)[churn_col].mean().sort_values(ascending=False)
            plt.figure(figsize=(8,4))
            grp.plot(kind='bar')
            plt.title(f'Churn rate por {c}')
            plt.ylabel('Churn rate (mean)')
            plt.show()


In [ ]:
# --- Análisis numérico (tarifas, tenure, gastos) ---
num_cols = df2.select_dtypes(include=[np.number]).columns.tolist()
num_cols

if churn_col:
    # Histogramas y boxplots para variables numéricas relevantes
    for c in ['monthlycharges','totalcharges','tenure','age','account_length']:
        if c in df2.columns:
            plt.figure()
            df2[c].dropna().hist(bins=30)
            plt.title(f'Histograma de {c}')
            plt.xlabel(c)
            plt.ylabel('Count')
            plt.show()
            
            plt.figure()
            df2.boxplot(column=c, by=churn_col)
            plt.suptitle('')
            plt.title(f'Boxplot de {c} por Churn')
            plt.show()

In [ ]:
# --- Correlaciones ---
if churn_col:
    corr = df2.select_dtypes(include=[np.number]).corr()
    # mostrar correlaciones con churn_col
    if churn_col in corr.columns:
        corr_with_churn = corr[churn_col].sort_values(ascending=False)
        print('Variables más correlacionadas con churn:\n', corr_with_churn.head(15))
    # Heatmap (matplotlib)
    plt.figure(figsize=(10,8))
    im = plt.imshow(corr, interpolation='nearest', aspect='auto')
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.title('Matriz de correlación (variables numéricas)')
    plt.tight_layout()
    plt.show()

## Conclusiones e insights (plantilla)

"
"Aquí comentarás los hallazgos principales después de ejecutar las celdas anteriores. Algunos ejemplos de insights que podrías encontrar:

"
"- **Tasa global de churn:** X%.
"
"- **Segmentos de alto riesgo:** clientes con contrato month-to-month, pagos electrónicos inestables, clientes con tenure bajo (< 6 meses), etc.
"
"- **Factores numéricos asociados al churn:** por ejemplo, aumento de `monthlycharges` o `totalcharges` correlaciona con mayor churn.

"
"## Recomendaciones estratégicas (plantilla)

"
"- Ofrecer descuentos o planes promocionales al renovar para clientes con tenure bajo.
"
"- Incentivar contratos a largo plazo con beneficios (reducción de tarifa, mejoras en servicio).
"
"- Revisar procesos de facturación y métodos de pago para reducir fricción.
"
"- Implementar un programa de retención (alerts, ofertas personalizadas) para segmentos con alto churn.

"
"---
"
"### Notas finales

"
"- Este notebook está listo para ejecutar en Google Colab. Si deseas, puedo adaptarlo para generar automáticamente un reporte en PDF o un dashboard (Streamlit / Dash).
")